In [1]:
!pip install -q sentence-transformers==3.2.1 ragas==0.2.2 peft==0.13.2 bitsandbytes==0.44.1 datasets==3.0.1 wandb==0.18.5 scipy==1.13.1 google-cloud-secret-manager

In [2]:
from datetime import datetime
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, BitsAndBytesConfig



In [3]:
try:
  from google.colab import userdata
  userdata.get("HF_TOKEN")
except userdata.SecretNotFoundError:
  print("HuggingFace Token not found, looking in caltech class project")
  from google.cloud import secretmanager
  import os
  client = secretmanager.SecretManagerServiceClient()
  response = client.access_secret_version(request={"name": "projects/240830225929/secrets/HF_TOKEN/versions/1"})
  os.environ["HF_TOKEN"] = response.payload.data.decode("UTF-8")


HuggingFace Token not found, looking in caltech class project


In [4]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import torch
from langchain_community.embeddings import HuggingFaceEmbeddings

# Model A From Step 1 in Lab01
base_model = "motherduckdb/DuckDB-NSQL-7B-v0.1"
modelA = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizerA = AutoTokenizer.from_pretrained(base_model)
embedding_modelA = HuggingFaceEmbeddings()


# Model B From Step 2 in Lab01
base_model = "codellama/CodeLlama-7b-hf"
modelB = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=BitsAndBytesConfig(load_in_8bit=True),
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizerB = AutoTokenizer.from_pretrained(base_model)
embedding_modelB = HuggingFaceEmbeddings()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-4-ce315739ad4c>:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_modelA = HuggingFaceEmbeddings()
<ipython-input-4-ce315739ad4c>:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_modelA = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-4-ce315739ad4c>:26: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_modelB = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
if torch.cuda.device_count() > 1:
    print(f"multi cuda devices #{torch.cuda.device_count()}")
    # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
    modelA.is_parallelizable = True
    modelA.model_parallel = True
    modelB.is_parallelizable = True
    modelB.model_parallel = True

multi cuda devices #2


In [6]:
from datasets import load_dataset
dataset = load_dataset("b-mc2/sql-create-context", split="train")
# train_dataset = dataset.train_test_split(test_size=0.1)["train"]
eval_dataset = dataset.train_test_split(test_size=0.1)["test"]


In [7]:

eval_dataset = eval_dataset.add_column("retrieved_contexts", [[],] * len(eval_dataset))

Flattening the indices:   0%|          | 0/7858 [00:00<?, ? examples/s]

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    #context_recall,
    #context_precision,
)
from ragas import evaluate
from ragas.metrics import context_entity_recall

pipe = pipeline(
    model=modelA,
    max_new_tokens=1024,
    do_sample=True,
    tokenizer=tokenizerA,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.1,
    repetition_penalty=1.1  # without this output begins repeating
)

evaluator = HuggingFacePipeline(pipeline=pipe)

# ragas
result = evaluate(
    dataset=eval_dataset,
    llm=evaluator,
    embeddings=embedding_modelA,
    metrics=[
        # context_entity_recall,
       # context_precision,
        faithfulness,
        answer_relevancy,
      #  context_recall,
    ],
)

<ipython-input-8-dd9e1b413309>:24: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  evaluator = HuggingFacePipeline(pipeline=pipe)


Evaluating:   0%|          | 0/15716 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
ERROR:ragas.executor:Exception raised in Job[7732]: RuntimeError(CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
)
ERROR:ragas.executor:Exception raised in Job[13967]: RuntimeError(CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORC